In [1]:
import pandas as pd
import numpy as np
import re
import requests

In [2]:
#read input sheets
Constituents = pd.read_excel('Copy of Data Import Assignment.xlsx', sheet_name = "Input Constituents")
Emails = pd.read_excel('Copy of Data Import Assignment.xlsx', sheet_name = "Input Emails")
Donations = pd.read_excel('Copy of Data Import Assignment.xlsx', sheet_name = "Input Donation History")

In [3]:
#check for id frequencies in each
unique_id_count = Constituents['Patron ID'].nunique()
unique_id_count

99

In [4]:
id_frequencies = Constituents['Patron ID'].value_counts()
id_frequencies

1288    2
8977    1
2481    1
3446    1
5967    1
       ..
4051    1
1089    1
4926    1
5767    1
7497    1
Name: Patron ID, Length: 99, dtype: int64

In [5]:
unique_id_count = Emails['Patron ID'].nunique()
unique_id_count

87

In [6]:
id_frequencies = Emails['Patron ID'].value_counts()
id_frequencies

8977    4
6522    4
3953    4
6366    4
3677    4
       ..
4107    1
4526    1
3973    1
7963    1
7497    1
Name: Patron ID, Length: 87, dtype: int64

In [7]:
unique_id_count = Donations['Patron ID'].nunique()
unique_id_count


90

In [8]:
id_frequencies = Donations['Patron ID'].value_counts()
id_frequencies

7963    4
1288    4
3140    3
9734    3
1348    3
       ..
3561    1
6846    1
6816    1
7040    1
6020    1
Name: Patron ID, Length: 90, dtype: int64

In [9]:
#Merging all the data into one single dataframe. Taking base as Constituents and considering only 99 patrons instead of 102(total number of unique patron ids combining all three data sheets), as the remaining 3 maybe present in Emails and Donations but they don't have a constituent profile

In [10]:
#make a copy of the base sheet(Constituents) without duplicates
cb = Constituents.drop_duplicates(subset="Patron ID").copy()

In [11]:
cb.shape

(99, 10)

In [12]:
cb["CB Constituent Type"] = np.where(
                            cb["Company"].notna() & (cb["Company"].str.strip() != "") & 
                            (cb["First Name"].isna() | (cb["First Name"].str.strip() == "")) & 
                            (cb["Last Name"].isna() | (cb["Last Name"].str.strip() == "")),
                            "Company",
                            "Person"
                            )

In [13]:
cb

,Patron ID,First Name,Last Name,Date Entered,Primary Email,Company,Salutation,Title,Tags,Gender,CB Constituent Type
0,8977,James,Baker,"Jan 19, 2020",walkerjeremy@long.org,NaN,Dr,NaN,Student Scholar,Unknown,Person
1,2798,Jessica,Sheppard,"Aug 20, 2019",tarasanchez@robinson.com,NaN,Mr,Graphic Designer,"Summer School 2016, Top Donor",Married,Person
2,3660,NaN,NaN,"Mar 20, 2018",wcochran@hotmail.com,Vaughn LLC,NaN,NaN,Summer School 2016,Married,Company
3,5966,Jared,Burns,"Apr 24, 2024",frazierjanice@gmaill.com,NaN,Dr,Administrative Assistant,"Pitch Perfect Staff, Major Donor 2021",Single,Person
4,5034,courtney,haney,"Jul 02, 2017",NaN,Lawrence LLC,Mr,NaN,"Pitch Perfect Volunteer, Pitch Perfect Staff",Unknown,Person
...,...,...,...,...,...,...,...,...,...,...,...
95,1348,Robert & Emily,Patton,"Dec 12, 2016",thomassamuel@gmaill.com,NaN,Mr. and Mrs.,NaN,NaN,Single,Person
96,1550,Laura,Harrison,2017-12-07 12:34:00,harrypatterson@yahoo.com,NaN,Mr. and Mrs.,NaN,"Major Donor 2021, Student Scholar, Board Member",Unknown,Person
97,3506,Jason,Moore,"Mar 27, 2015",traciemcpherson@palmer.com,Used to work here.,Mr. and Mrs.,NaN,NaN,Unknown,Person
98,3141,jennifer,JUAREZ,"Jan 30, 2022",xhubbard@garcia.biz,NaN,NaN,NaN,Student Scholar,Married,Person


In [14]:
cb["CB First Name"] = np.where(
                      cb["CB Constituent Type"] == "Person",
                      cb["First Name"].str.capitalize(),
                      "")

In [15]:
cb["CB Last Name"] = np.where(
                      cb["CB Constituent Type"] == "Person",
                      cb["Last Name"].str.capitalize(),
                      "")

In [16]:
cb["CB Company Name"] = np.where(
                        cb["CB Constituent Type"] == "Company",
                        cb["Company"],
                        "")

In [17]:
cb

,Patron ID,First Name,Last Name,Date Entered,Primary Email,Company,Salutation,Title,Tags,Gender,CB Constituent Type,CB First Name,CB Last Name,CB Company Name
0,8977,James,Baker,"Jan 19, 2020",walkerjeremy@long.org,NaN,Dr,NaN,Student Scholar,Unknown,Person,James,Baker,
1,2798,Jessica,Sheppard,"Aug 20, 2019",tarasanchez@robinson.com,NaN,Mr,Graphic Designer,"Summer School 2016, Top Donor",Married,Person,Jessica,Sheppard,
2,3660,NaN,NaN,"Mar 20, 2018",wcochran@hotmail.com,Vaughn LLC,NaN,NaN,Summer School 2016,Married,Company,,,Vaughn LLC
3,5966,Jared,Burns,"Apr 24, 2024",frazierjanice@gmaill.com,NaN,Dr,Administrative Assistant,"Pitch Perfect Staff, Major Donor 2021",Single,Person,Jared,Burns,
4,5034,courtney,haney,"Jul 02, 2017",NaN,Lawrence LLC,Mr,NaN,"Pitch Perfect Volunteer, Pitch Perfect Staff",Unknown,Person,Courtney,Haney,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1348,Robert & Emily,Patton,"Dec 12, 2016",thomassamuel@gmaill.com,NaN,Mr. and Mrs.,NaN,NaN,Single,Person,Robert & emily,Patton,
96,1550,Laura,Harrison,2017-12-07 12:34:00,harrypatterson@yahoo.com,NaN,Mr. and Mrs.,NaN,"Major Donor 2021, Student Scholar, Board Member",Unknown,Person,Laura,Harrison,
97,3506,Jason,Moore,"Mar 27, 2015",traciemcpherson@palmer.com,Used to work here.,Mr. and Mrs.,NaN,NaN,Unknown,Person,Jason,Moore,
98,3141,jennifer,JUAREZ,"Jan 30, 2022",xhubbard@garcia.biz,NaN,NaN,NaN,Student Scholar,Married,Person,Jennifer,Juarez,


In [18]:
cb["CB Created At"] = pd.to_datetime(cb["Date Entered"])

In [19]:
cb

,Patron ID,First Name,Last Name,Date Entered,Primary Email,Company,Salutation,Title,Tags,Gender,CB Constituent Type,CB First Name,CB Last Name,CB Company Name,CB Created At
0,8977,James,Baker,"Jan 19, 2020",walkerjeremy@long.org,NaN,Dr,NaN,Student Scholar,Unknown,Person,James,Baker,,2020-01-19 00:00:00
1,2798,Jessica,Sheppard,"Aug 20, 2019",tarasanchez@robinson.com,NaN,Mr,Graphic Designer,"Summer School 2016, Top Donor",Married,Person,Jessica,Sheppard,,2019-08-20 00:00:00
2,3660,NaN,NaN,"Mar 20, 2018",wcochran@hotmail.com,Vaughn LLC,NaN,NaN,Summer School 2016,Married,Company,,,Vaughn LLC,2018-03-20 00:00:00
3,5966,Jared,Burns,"Apr 24, 2024",frazierjanice@gmaill.com,NaN,Dr,Administrative Assistant,"Pitch Perfect Staff, Major Donor 2021",Single,Person,Jared,Burns,,2024-04-24 00:00:00
4,5034,courtney,haney,"Jul 02, 2017",NaN,Lawrence LLC,Mr,NaN,"Pitch Perfect Volunteer, Pitch Perfect Staff",Unknown,Person,Courtney,Haney,,2017-07-02 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1348,Robert & Emily,Patton,"Dec 12, 2016",thomassamuel@gmaill.com,NaN,Mr. and Mrs.,NaN,NaN,Single,Person,Robert & emily,Patton,,2016-12-12 00:00:00
96,1550,Laura,Harrison,2017-12-07 12:34:00,harrypatterson@yahoo.com,NaN,Mr. and Mrs.,NaN,"Major Donor 2021, Student Scholar, Board Member",Unknown,Person,Laura,Harrison,,2017-12-07 12:34:00
97,3506,Jason,Moore,"Mar 27, 2015",traciemcpherson@palmer.com,Used to work here.,Mr. and Mrs.,NaN,NaN,Unknown,Person,Jason,Moore,,2015-03-27 00:00:00
98,3141,jennifer,JUAREZ,"Jan 30, 2022",xhubbard@garcia.biz,NaN,NaN,NaN,Student Scholar,Married,Person,Jennifer,Juarez,,2022-01-30 00:00:00


In [20]:
#standardize all emails
Emails["Email"] = (Emails["Email"].str.lower().str.strip())

In [21]:
emails_agg = Emails.groupby("Patron ID")["Email"] \
                   .unique() \
                   .apply(list) \
                   .reset_index()

emails_agg["CB Email 1 (Standardized)"] = emails_agg["Email"].str.get(0).fillna("")
emails_agg["CB Email 2 (Standardized)"] = emails_agg["Email"].str.get(1).fillna("")

In [22]:
emails_agg

,Patron ID,Email,CB Email 1 (Standardized),CB Email 2 (Standardized)
0,1089,"[samantha08@hotmail.com, andrewsrandy@howard.c...",samantha08@hotmail.com,andrewsrandy@howard.com
1,1102,"[warrendaniel@gmail.com, xwilson@olsen-morgan....",warrendaniel@gmail.com,xwilson@olsen-morgan.org
2,1288,"[ellendavis@smith.com, john56@stone.org]",ellendavis@smith.com,john56@stone.org
3,1348,[thomassamuel@gmaill.com],thomassamuel@gmaill.com,
4,1459,"[millersherry@gmaill.com, millernicole@stanley...",millersherry@gmaill.com,millernicole@stanley-harris.com
...,...,...,...,...
82,9734,"[victoria10@gmal.com, hansonchristine@hotmail....",victoria10@gmal.com,hansonchristine@hotmail.com
83,9821,"[klinejose@gmail.com, brittanybenson@galvan.co...",klinejose@gmail.com,brittanybenson@galvan.com
84,9827,"[ryan82@yahoo.com, mark55@yahoo.com, clarkdebo...",ryan82@yahoo.com,mark55@yahoo.com
85,9859,"[michael43@hall-lee.com, harperluke@gmail.com,...",michael43@hall-lee.com,harperluke@gmail.com


In [23]:
emails_agg = emails_agg.drop(columns="Email")


In [24]:
emails_agg

,Patron ID,CB Email 1 (Standardized),CB Email 2 (Standardized)
0,1089,samantha08@hotmail.com,andrewsrandy@howard.com
1,1102,warrendaniel@gmail.com,xwilson@olsen-morgan.org
2,1288,ellendavis@smith.com,john56@stone.org
3,1348,thomassamuel@gmaill.com,
4,1459,millersherry@gmaill.com,millernicole@stanley-harris.com
...,...,...,...
82,9734,victoria10@gmal.com,hansonchristine@hotmail.com
83,9821,klinejose@gmail.com,brittanybenson@galvan.com
84,9827,ryan82@yahoo.com,mark55@yahoo.com
85,9859,michael43@hall-lee.com,harperluke@gmail.com


In [25]:
#Mapping the title to the right format of Salutation
titles = {"Mr": "Mr.", "Mrs": "Mrs.", "Ms": "Ms.","Dr": "Dr."}
cb["CB Title"] = cb["Salutation"].map(titles).fillna("")

In [26]:
cb

,Patron ID,First Name,Last Name,Date Entered,Primary Email,Company,Salutation,Title,Tags,Gender,CB Constituent Type,CB First Name,CB Last Name,CB Company Name,CB Created At,CB Title
0,8977,James,Baker,"Jan 19, 2020",walkerjeremy@long.org,NaN,Dr,NaN,Student Scholar,Unknown,Person,James,Baker,,2020-01-19 00:00:00,Dr.
1,2798,Jessica,Sheppard,"Aug 20, 2019",tarasanchez@robinson.com,NaN,Mr,Graphic Designer,"Summer School 2016, Top Donor",Married,Person,Jessica,Sheppard,,2019-08-20 00:00:00,Mr.
2,3660,NaN,NaN,"Mar 20, 2018",wcochran@hotmail.com,Vaughn LLC,NaN,NaN,Summer School 2016,Married,Company,,,Vaughn LLC,2018-03-20 00:00:00,
3,5966,Jared,Burns,"Apr 24, 2024",frazierjanice@gmaill.com,NaN,Dr,Administrative Assistant,"Pitch Perfect Staff, Major Donor 2021",Single,Person,Jared,Burns,,2024-04-24 00:00:00,Dr.
4,5034,courtney,haney,"Jul 02, 2017",NaN,Lawrence LLC,Mr,NaN,"Pitch Perfect Volunteer, Pitch Perfect Staff",Unknown,Person,Courtney,Haney,,2017-07-02 00:00:00,Mr.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1348,Robert & Emily,Patton,"Dec 12, 2016",thomassamuel@gmaill.com,NaN,Mr. and Mrs.,NaN,NaN,Single,Person,Robert & emily,Patton,,2016-12-12 00:00:00,
96,1550,Laura,Harrison,2017-12-07 12:34:00,harrypatterson@yahoo.com,NaN,Mr. and Mrs.,NaN,"Major Donor 2021, Student Scholar, Board Member",Unknown,Person,Laura,Harrison,,2017-12-07 12:34:00,
97,3506,Jason,Moore,"Mar 27, 2015",traciemcpherson@palmer.com,Used to work here.,Mr. and Mrs.,NaN,NaN,Unknown,Person,Jason,Moore,,2015-03-27 00:00:00,
98,3141,jennifer,JUAREZ,"Jan 30, 2022",xhubbard@garcia.biz,NaN,NaN,NaN,Student Scholar,Married,Person,Jennifer,Juarez,,2022-01-30 00:00:00,


In [27]:
response = requests.get("https://6719768f7fc4c5ff8f4d84f1.mockapi.io/api/v1/tags")
tag_map = {t["name"]: t["mapped_name"] for t in response.json()}

In [28]:
def format_tags(tags):
    cleaned_tags = []

    for t in tags:
        t = t.strip()
        if not t:
            continue

        cleaned_tags.append(tag_map.get(t, t))

    return ", ".join(set(cleaned_tags))

In [29]:
cb["CB Tags"] = (cb["Tags"].fillna("").str.split(",").apply(format_tags))

In [30]:
cb

,Patron ID,First Name,Last Name,Date Entered,Primary Email,Company,Salutation,Title,Tags,Gender,CB Constituent Type,CB First Name,CB Last Name,CB Company Name,CB Created At,CB Title,CB Tags
0,8977,James,Baker,"Jan 19, 2020",walkerjeremy@long.org,NaN,Dr,NaN,Student Scholar,Unknown,Person,James,Baker,,2020-01-19 00:00:00,Dr.,Student Scholar
1,2798,Jessica,Sheppard,"Aug 20, 2019",tarasanchez@robinson.com,NaN,Mr,Graphic Designer,"Summer School 2016, Top Donor",Married,Person,Jessica,Sheppard,,2019-08-20 00:00:00,Mr.,"Major Donor, Summer 2016"
2,3660,NaN,NaN,"Mar 20, 2018",wcochran@hotmail.com,Vaughn LLC,NaN,NaN,Summer School 2016,Married,Company,,,Vaughn LLC,2018-03-20 00:00:00,,Summer 2016
3,5966,Jared,Burns,"Apr 24, 2024",frazierjanice@gmaill.com,NaN,Dr,Administrative Assistant,"Pitch Perfect Staff, Major Donor 2021",Single,Person,Jared,Burns,,2024-04-24 00:00:00,Dr.,"Pitch Perfect, Major Donor"
4,5034,courtney,haney,"Jul 02, 2017",NaN,Lawrence LLC,Mr,NaN,"Pitch Perfect Volunteer, Pitch Perfect Staff",Unknown,Person,Courtney,Haney,,2017-07-02 00:00:00,Mr.,Pitch Perfect
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1348,Robert & Emily,Patton,"Dec 12, 2016",thomassamuel@gmaill.com,NaN,Mr. and Mrs.,NaN,NaN,Single,Person,Robert & emily,Patton,,2016-12-12 00:00:00,,
96,1550,Laura,Harrison,2017-12-07 12:34:00,harrypatterson@yahoo.com,NaN,Mr. and Mrs.,NaN,"Major Donor 2021, Student Scholar, Board Member",Unknown,Person,Laura,Harrison,,2017-12-07 12:34:00,,"Major Donor, Student Scholar, Board Member"
97,3506,Jason,Moore,"Mar 27, 2015",traciemcpherson@palmer.com,Used to work here.,Mr. and Mrs.,NaN,NaN,Unknown,Person,Jason,Moore,,2015-03-27 00:00:00,,
98,3141,jennifer,JUAREZ,"Jan 30, 2022",xhubbard@garcia.biz,NaN,NaN,NaN,Student Scholar,Married,Person,Jennifer,Juarez,,2022-01-30 00:00:00,,Student Scholar


In [31]:
cb["CB Background Information"] = ""

In [32]:
# Job title part
cb.loc[cb["Title"].notna() & (cb["Title"].str.strip() !=""),"CB Background Information"]="Job Title: " + cb["Title"]

In [33]:
# Marital status part (based on 'Gender' column)
status = cb["Gender"].notna() & (cb["Gender"].str.strip() != "")

In [34]:
cb.loc[status, "CB Background Information"] = (
    cb.loc[status, "CB Background Information"].where(cb.loc[status,"CB Background Information"] == "",
           cb.loc[status, "CB Background Information"] + "; ")
    + "Marital Status: " + cb["Gender"]
)

In [35]:
cb["CB Background Information"]

0                               Marital Status: Unknown
1     Job Title: Graphic Designer; Marital Status: M...
2                               Marital Status: Married
3     Job Title: Administrative Assistant; Marital S...
4                               Marital Status: Unknown
                            ...                        
95                               Marital Status: Single
96                              Marital Status: Unknown
97                              Marital Status: Unknown
98                              Marital Status: Married
99                              Marital Status: Unknown
Name: CB Background Information, Length: 99, dtype: object

In [37]:
cb

,Patron ID,First Name,Last Name,Date Entered,Primary Email,Company,Salutation,Title,Tags,Gender,CB Constituent Type,CB First Name,CB Last Name,CB Company Name,CB Created At,CB Title,CB Tags,CB Background Information
0,8977,James,Baker,"Jan 19, 2020",walkerjeremy@long.org,NaN,Dr,NaN,Student Scholar,Unknown,Person,James,Baker,,2020-01-19 00:00:00,Dr.,Student Scholar,Marital Status: Unknown
1,2798,Jessica,Sheppard,"Aug 20, 2019",tarasanchez@robinson.com,NaN,Mr,Graphic Designer,"Summer School 2016, Top Donor",Married,Person,Jessica,Sheppard,,2019-08-20 00:00:00,Mr.,"Major Donor, Summer 2016",Job Title: Graphic Designer; Marital Status: M...
2,3660,NaN,NaN,"Mar 20, 2018",wcochran@hotmail.com,Vaughn LLC,NaN,NaN,Summer School 2016,Married,Company,,,Vaughn LLC,2018-03-20 00:00:00,,Summer 2016,Marital Status: Married
3,5966,Jared,Burns,"Apr 24, 2024",frazierjanice@gmaill.com,NaN,Dr,Administrative Assistant,"Pitch Perfect Staff, Major Donor 2021",Single,Person,Jared,Burns,,2024-04-24 00:00:00,Dr.,"Pitch Perfect, Major Donor",Job Title: Administrative Assistant; Marital S...
4,5034,courtney,haney,"Jul 02, 2017",NaN,Lawrence LLC,Mr,NaN,"Pitch Perfect Volunteer, Pitch Perfect Staff",Unknown,Person,Courtney,Haney,,2017-07-02 00:00:00,Mr.,Pitch Perfect,Marital Status: Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1348,Robert & Emily,Patton,"Dec 12, 2016",thomassamuel@gmaill.com,NaN,Mr. and Mrs.,NaN,NaN,Single,Person,Robert & emily,Patton,,2016-12-12 00:00:00,,,Marital Status: Single
96,1550,Laura,Harrison,2017-12-07 12:34:00,harrypatterson@yahoo.com,NaN,Mr. and Mrs.,NaN,"Major Donor 2021, Student Scholar, Board Member",Unknown,Person,Laura,Harrison,,2017-12-07 12:34:00,,"Major Donor, Student Scholar, Board Member",Marital Status: Unknown
97,3506,Jason,Moore,"Mar 27, 2015",traciemcpherson@palmer.com,Used to work here.,Mr. and Mrs.,NaN,NaN,Unknown,Person,Jason,Moore,,2015-03-27 00:00:00,,,Marital Status: Unknown
98,3141,jennifer,JUAREZ,"Jan 30, 2022",xhubbard@garcia.biz,NaN,NaN,NaN,Student Scholar,Married,Person,Jennifer,Juarez,,2022-01-30 00:00:00,,Student Scholar,Marital Status: Married


In [39]:
#donation relevant queries
Donations["Donation Date"] = pd.to_datetime(Donations["Donation Date"])

In [42]:
# Aggregate donations amount and dates
Donations_agg = (Donations.sort_values("Donation Date").groupby("Patron ID", as_index=False)
                .agg(
                lifetime_amount=("Donation Amount", "sum"),
                recent_date=("Donation Date", "max"),
                recent_amount=("Donation Amount", "last")
                ))

In [43]:
#lifetime donation amount
Donations_agg["CB Lifetime Donation Amount"] = ("$" + Donations_agg["lifetime_amount"].round(2).astype(str))

In [44]:
# Most recent donation date
Donations_agg["CB Most Recent Donation Date"] = Donations_agg["recent_date"]

In [45]:
#most recent donation amount
Donations_agg["CB Most Recent Donation Amount"] = ("$" + Donations_agg["recent_amount"].round(2).astype(str))


In [54]:
output1 = cb.merge(emails_agg, on = 'Patron ID', how='left').merge(Donations_agg, on = 'Patron ID', how = 'left')

In [55]:
output1 = output1.drop(columns = ['lifetime_amount', 'recent_date', 'recent_amount', 'First Name', 'Last Name', 'Date Entered',
                                 'Primary Email', 'Company', 'Salutation','Title','Tags', 'Gender'])

In [56]:
output1.columns

Index(['Patron ID', 'CB Constituent Type', 'CB First Name', 'CB Last Name',
       'CB Company Name', 'CB Created At', 'CB Title', 'CB Tags',
       'CB Background Information', 'CB Email 1 (Standardized)',
       'CB Email 2 (Standardized)', 'CB Lifetime Donation Amount',
       'CB Most Recent Donation Date', 'CB Most Recent Donation Amount'],
      dtype='object')

In [57]:
output1 = output1.rename(columns = {"Patron ID": "CB Constituent ID"})

In [58]:
output1.columns

Index(['CB Constituent ID', 'CB Constituent Type', 'CB First Name',
       'CB Last Name', 'CB Company Name', 'CB Created At', 'CB Title',
       'CB Tags', 'CB Background Information', 'CB Email 1 (Standardized)',
       'CB Email 2 (Standardized)', 'CB Lifetime Donation Amount',
       'CB Most Recent Donation Date', 'CB Most Recent Donation Amount'],
      dtype='object')

In [61]:
assert output1["CB Constituent ID"].is_unique
#pass

In [63]:
output1.to_csv("Output Cuebox Constituents.csv", index = False)

In [70]:
#Tags
#The idea is to make all values from 'CB Tags' column into a list and just count value frequencies in the list
all_tags = []

In [71]:
for tags in output1['CB Tags'].dropna():
    #split by comma and strip whitespace from each tag
    tag_list = [tag.strip() for tag in tags.split(',')]
    all_tags.extend(tag_list)

In [82]:
#count frequencies
tag_counts={}
for tag in all_tags:
    if tag != '': # to skip empty strings
        if tag in tag_counts:
            #tag already exists, increment its count
            tag_counts[tag] += 1
        else:
            #tag is new, initialize its count to 1
            tag_counts[tag] = 1

In [80]:
output2 = pd.DataFrame([{'CB Tag Name': tag, 'CB Tag Count': count} for tag, count in tag_counts.items()])

In [83]:
output2.to_csv('Output CueBox Tags.csv', index = False)